<a href="https://colab.research.google.com/github/mighty-mass/ai-engineer-course/blob/main/ML%20Models/TropicTaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>